In [ ]:
from fastai.vision.all import *

# Dataset
path = untar_data(URLs.PETS)
path_img = path / "images"

def is_cat(x): return Path(x).name[0].isupper()

dls = ImageDataLoaders.from_name_func(
    path_img,
    get_image_files(path_img),
    valid_pct=0.2,
    seed=42,
    label_func=is_cat,
    item_tfms=Resize(224)
)

In [ ]:
learn = vision_learner(dls, resnet34, metrics=accuracy)

# EXPERIMENT: Use 5 epochs and a custom learning rate
learn.fine_tune(5, base_lr=1e-4)

In [ ]:
learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(3, base_lr=1e-1)  # Intentionally too high

## Hyperparameter Experimentation: Epochs and Learning Rate

### Objective
The goal of this experiment was to analyze the effect of modifying training hyperparameters—specifically the number of epochs and the learning rate—on model performance and training stability.

---

### Baseline (3 epochs, default LR = 3e-3)

| Metric        | Final Value |
|---------------|-------------|
| Accuracy      | 0.9959      |
| Valid Loss    | 0.0093      |

Using the fastai default learning rate (`3e-3`) with 3 epochs, the model achieved high accuracy (~99.6%) and very low validation loss. This establishes a strong baseline for comparison.

---

### Experiment 1: Lower LR (1e-4), More Epochs (5)

| Metric        | Final Value |
|---------------|-------------|
| Accuracy      | 0.9932      |
| Valid Loss    | 0.0160      |

Despite a longer training period (5 epochs), reducing the learning rate led to **slower convergence**. While the model maintained high accuracy, it underperformed slightly compared to the baseline in both accuracy and loss. This reflects the tradeoff of a conservative learning rate: **greater training stability at the cost of learning speed**. Interestingly, the model achieved ~99.3% accuracy by epoch 2, with marginal improvement thereafter, suggesting **early stopping** could have been beneficial.

---

### Experiment 2: High LR (1e-1), 3 Epochs

| Metric        | Final Value |
|---------------|-------------|
| Accuracy      | 0.9756      |
| Valid Loss    | 0.0721      |

Using an aggressively high learning rate (`1e-1`) destabilized training. The model exhibited:
- Extremely high validation loss in epoch 0 (`3.68`)
- Erratic loss curves
- Delayed accuracy improvements

Despite eventually recovering to ~97.6% accuracy, this configuration underperformed both in stability and overall validation performance. The result aligns with expectations: **a learning rate too high overshoots optimal weights**, leading to poor generalization and instability.

---

### Takeaways

- The **default fastai setting (3e-3, 3 epochs)** yielded the best generalization and fastest convergence.
- A **lower LR (1e-4)** can be beneficial for highly noisy data or fine-grained tuning, but may not outperform the baseline without increased epochs or regularization.
- A **high LR (1e-1)** is unsafe without a robust LR scheduler or gradient clipping—it leads to high variance and risk of divergence.
- **Hyperparameter tuning should balance convergence speed, stability, and validation performance**, and decisions should be guided by empirical trends in loss curves and accuracy saturation.